# Introduction to tf.keras

In [ ]:
import sys
import tensorflow as tf

import numpy as np
%matplotlib inline
import random
import matplotlib.pyplot as plt

In [ ]:
# Python version 3.5 or 3.6
assert sys.version_info >= (3, 5)
assert sys.version_info < (3, 7)
# Tensorflow 2.0
assert tf.__version__ >= "2.0"

We will take a look at a first concrete example of a neural network, which makes use of `tf.keras` to learn to classify 
images. 

The problem we are trying to solve here is to classify RGB images (32 pixels by 32 pixels), into their 10 categories (_airplane_, _automobile_, _bird_, _cat_, _deer_, _dog_, _frog_, _horse_, _ship_, _truck_). The dataset we will use is the CIFAR10 dataset, a classic dataset in the machine learning community.

# Input Data Management

## Download the dataset

The CIFAR10 dataset comes pre-loaded in Keras, in the form of a set of four Numpy arrays.

Documentation : https://www.tensorflow.org/api_docs/python/tf/keras/datasets/cifar10

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

## Visualize the data

Let's print the shapes of the datasets

In [ ]:
print("Train images shape : {}".format(train_images.shape))
print("Train labels shape : {}".format(train_labels.shape))
print("Test images shape : {}".format(test_images.shape))
print("Test labels shape : {}".format(test_labels.shape))

Let's see how the images look. This function shows a random example along with it's corresponding label.

In [ ]:
i = random.randint(0, 100)

print("Label: %s" % train_labels[i])
plt.imshow(train_images[i], cmap='gray')

Thats a little blurry !

Our workflow will be as follow: first we will present our neural network with the training data, `train_images` and `train_labels`. The network will then learn to associate images and labels. Finally, we will ask the network to produce predictions for `test_images`, and we will verify if these predictions match the labels from `test_labels`.

## Reformat the data

The labels have values from 0 to 9, but neural networks need instead to have access to a one-hot encoded vector (a vector of length 10 with all zeros but one at the index of the label).

Documentation : https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical

In [ ]:
NUM_CAT = 10
print("Before", train_labels[0]) # The format of the labels before conversion
train_cat  = tf.keras.utils.to_categorical(train_labels, NUM_CAT)
print("After", train_labels[0]) # The format of the labels after conversion
test_cat = tf.keras.utils.to_categorical(test_labels, NUM_CAT)

Train and test images values are integers from 1 to 255. We need to convert then into floats with values from 0 to 1.

> <div class="mark">Cast the images to floats and convert the pixel values from integers between 0 and 255 to floats between 0 and 1</div><i class="fa fa-lightbulb-o "></i>

Hint : Use the `astype` numpy function

In [ ]:
train_images_01 = train_images.astype(np.float32) / 255
test_images_01 = test_images.astype(np.float32) / 255

We will apply image augmentation in the input processing pipeline. Here are the transformations we can apply :
- `random_flip_left_right` : Flip the image
- `random_brightness` : Randomly modify image brightness (use the `max_delta`, for example with a value of 32.0/255)
- `random_saturation` : Randomly modify image brightness (use the `lower` and `upper` arguments, for example with values 0.5 and 1.5 respectively)

Finally, in order to make sure the image values are still in the [0,1] interval, use the `clip_by_value` function.

Remember to only apply these transformations to the train set, not the test set, as we want the test images to be the real ones.

> <div class="mark">Create the _train_preprocess function to apply image augmentation.</div><i class="fa fa-lightbulb-o "></i>

Documentation : 
- TODO

In [ ]:
def _train_preprocess(image, label):
    # Flip image
    image = tf.image.random_flip_left_right(image)
    # Random Brightness
    image = tf.image.random_brightness(image, max_delta=32.0 / 255.0)
    # Random Saturation
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
    # Clip values between 0 and 1
    image = tf.clip_by_value(image, 0.0, 1.0)

    return image, label

In [ ]:
def _train_preprocess(image, label):
    # Flip image
    image = # TODO
    # Random Brightness
    image = # TODO
    # Random Saturation
    image = # TODO
    # Clip values between 0 and 1
    image = # TODO

    return image, label

> <div class="mark">Add the function to the training input pipeline</div><i class="fa fa-lightbulb-o "></i>

> TODO

# Model Management

The core building block of neural networks is the "layer", a data-processing module which you can conceive as a "filter" for data. Some data comes in, and comes out in a more useful form. Precisely, layers extract _representations_ out of the data fed into them -- hopefully representations that are more meaningful for the problem at hand. Most of deep learning really consists of chaining together simple layers which will implement a form of progressive "data distillation". A deep learning model is like a sieve for data processing, made of a succession of increasingly refined data filters -- the "layers".

The last layer is a 10-way "softmax" layer, which means it will return an array of 10 probability scores (summing to 1). Each score will be the probability that the current image belongs to one of our 10 classes.

## Build the model

Our Neural Network will contain the following building blocks : 
- Conv2D Layer : 16 filters, (3, 3) kernel, relu activation, input shape (32, 32, 3)
- MaxPooling2D : pool size (2, 2)
- Conv2D Layer : 32 filters, (3, 3) kernel, relu activation
- MaxPooling2D : pool size (2, 2)
- Flatten layer
- Dense Layer : 10 neurons, softmax activation

Documentation : 
- https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential
- https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D
- https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D
- https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten
- https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, activation='softmax'))

To make our network ready for training, we need to pick three more things, as part of "compilation" step:

* A loss function: the is how the network will be able to measure how good a job it is doing on its training data, and thus how it will be able to steer itself in the right direction.
* An optimizer: this is the mechanism through which the network will update itself based on the data it sees and its loss function.
* Metrics to monitor during training and testing. Here we will only care about accuracy (the fraction of the images that were correctly classified).

You will implement the following compilation step for your Neural Network : 
- "adam" optimizer
- "categorical_crossentropy" loss
- metric : "accuracy"

Documentation : https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#compile

In [ ]:
optimizer = tf.optimizers.Adam()

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

Summarize the model

In [ ]:
model.summary()

## Train the model

We are now ready to train our network, which in Keras is done via a call to the `fit` method of the network: 
we "fit" the model to its training data.

You will fit the network with the following configurations :
- 5 epochs (passes on the whole dataset)
- batchs of 128 images

Documentation : https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#fit

You will also add a callback for launching TensorBoard to observe how the training is performing.

In [ ]:
tensorboard = tf.keras.callbacks.TensorBoard(log_dir='./tensorboard/tf_keras', histogram_freq=0, write_graph=True, write_images=True)

In [ ]:
model.fit(train_images_01, train_cat, epochs=5, batch_size=32, callbacks=[tensorboard])

Two quantities are being displayed during training: the "loss" of the network over the training data, and the accuracy of the network over the training data.

# Model Performance Evaluation

Now let's check that our model performs well on the test set too.

You can do this by calling the `evaluate` method of your network.

Documentation : https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#evaluate

In [ ]:
loss, accuracy = model.evaluate(test_images_01, test_cat)

print('Test accuracy: %.2f' % (accuracy))